# Day 2 (4-6h): 캡스톤 실습 & 운영 계획

## 종합 과제: 스마트 팩토리 품질 예측 시스템 구축

### 시나리오
전자부품 제조 공장에서 품질 불량을 사전에 예측하고, 공정을 최적화하는 AI 시스템을 구축합니다.
- 실시간 센서 데이터 수집
- 불량 예측 모델 개발
- 예지보전 시스템 통합
- 대시보드 및 알람 시스템 구현

In [ ]:
# 필요 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import json
import pickle
import warnings
warnings.filterwarnings('ignore')

# ML/DL 라이브러리
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.metrics import classification_report, confusion_matrix
import torch
import torch.nn as nn

print("Libraries loaded successfully")

## 1. 통합 데이터 파이프라인 구축

In [ ]:
class SmartFactoryDataPipeline:
    """스마트 팩토리 데이터 파이프라인"""
    
    def __init__(self):
        self.data_sources = {
            'sensors': ['temperature', 'pressure', 'humidity', 'vibration', 'current'],
            'process': ['speed', 'feed_rate', 'tool_wear', 'cycle_time'],
            'quality': ['dimension', 'surface_finish', 'hardness']
        }
        self.data_buffer = []
        self.processed_data = None
        
    def generate_sensor_data(self, n_samples=1000, anomaly_rate=0.05):
        """센서 데이터 시뮬레이션"""
        np.random.seed(42)
        
        # 정상 데이터 생성
        data = {
            'timestamp': pd.date_range(start='2024-01-01', periods=n_samples, freq='1Min'),
            'temperature': np.random.normal(75, 5, n_samples),
            'pressure': np.random.normal(100, 10, n_samples),
            'humidity': np.random.normal(50, 5, n_samples),
            'vibration': np.random.normal(2, 0.5, n_samples),
            'current': np.random.normal(15, 2, n_samples),
            'speed': np.random.normal(1500, 50, n_samples),
            'feed_rate': np.random.normal(100, 10, n_samples),
            'tool_wear': np.random.uniform(0, 1, n_samples),
            'cycle_time': np.random.normal(60, 5, n_samples)
        }
        
        # 품질 데이터
        data['dimension'] = 10 + 0.01 * data['temperature'] - 0.005 * data['pressure'] + np.random.normal(0, 0.1, n_samples)
        data['surface_finish'] = 1.5 - 0.001 * data['speed'] + 0.5 * data['tool_wear'] + np.random.normal(0, 0.05, n_samples)
        data['hardness'] = 60 + 0.1 * data['temperature'] - 0.02 * data['cycle_time'] + np.random.normal(0, 1, n_samples)
        
        # 이상 패턴 주입
        n_anomalies = int(n_samples * anomaly_rate)
        anomaly_indices = np.random.choice(n_samples, n_anomalies, replace=False)
        
        # 다양한 이상 패턴
        for idx in anomaly_indices:
            anomaly_type = np.random.choice(['temperature', 'vibration', 'tool_wear', 'multi'])
            
            if anomaly_type == 'temperature':
                data['temperature'][idx] += np.random.choice([-20, 20])
            elif anomaly_type == 'vibration':
                data['vibration'][idx] *= np.random.uniform(3, 5)
            elif anomaly_type == 'tool_wear':
                data['tool_wear'][idx] = min(1.0, data['tool_wear'][idx] * 2)
            else:  # multi
                data['temperature'][idx] += 15
                data['vibration'][idx] *= 2
                data['pressure'][idx] -= 20
        
        # 불량 라벨 생성 (규격 기반)
        defects = (
            (np.abs(data['dimension'] - 10) > 0.5) |
            (data['surface_finish'] > 2.0) |
            (data['hardness'] < 55) | (data['hardness'] > 65)
        )
        
        data['defect'] = defects.astype(int)
        data['anomaly'] = np.zeros(n_samples)
        data['anomaly'][anomaly_indices] = 1
        
        return pd.DataFrame(data)
    
    def extract_features(self, df, window_size=10):
        """특징 추출 (통계적 특징 + 시계열 특징)"""
        features = df.copy()
        
        # Rolling window 특징
        sensor_cols = ['temperature', 'pressure', 'vibration', 'current']
        
        for col in sensor_cols:
            features[f'{col}_rolling_mean'] = df[col].rolling(window=window_size, min_periods=1).mean()
            features[f'{col}_rolling_std'] = df[col].rolling(window=window_size, min_periods=1).std()
            features[f'{col}_rolling_max'] = df[col].rolling(window=window_size, min_periods=1).max()
            features[f'{col}_rolling_min'] = df[col].rolling(window=window_size, min_periods=1).min()
        
        # 변화율 특징
        for col in sensor_cols:
            features[f'{col}_diff'] = df[col].diff()
            features[f'{col}_pct_change'] = df[col].pct_change()
        
        # 상호작용 특징
        features['temp_pressure_ratio'] = features['temperature'] / (features['pressure'] + 1)
        features['vibration_current_product'] = features['vibration'] * features['current']
        features['efficiency'] = features['speed'] / (features['cycle_time'] + 1)
        
        # NaN 처리
        features = features.fillna(method='ffill').fillna(0)
        
        return features
    
    def create_train_test_split(self, df, test_size=0.2):
        """시계열을 고려한 학습/테스트 분할"""
        # 시간 순서 유지
        split_index = int(len(df) * (1 - test_size))
        
        train_df = df.iloc[:split_index]
        test_df = df.iloc[split_index:]
        
        return train_df, test_df

# 데이터 파이프라인 실행
pipeline = SmartFactoryDataPipeline()

# 데이터 생성
print("Generating manufacturing data...")
raw_data = pipeline.generate_sensor_data(n_samples=2000, anomaly_rate=0.05)

# 특징 추출
print("Extracting features...")
feature_data = pipeline.extract_features(raw_data)

# 데이터 분할
train_data, test_data = pipeline.create_train_test_split(feature_data)

print(f"\nData Pipeline Summary:")
print(f"  Total samples: {len(raw_data)}")
print(f"  Features: {len(feature_data.columns)}")
print(f"  Training samples: {len(train_data)}")
print(f"  Test samples: {len(test_data)}")
print(f"  Defect rate: {raw_data['defect'].mean():.2%}")
print(f"  Anomaly rate: {raw_data['anomaly'].mean():.2%}")

## 2. 통합 AI 모델 시스템

In [ ]:
class IntegratedAISystem:
    """통합 AI 시스템 (품질 예측 + 이상 탐지 + 예지보전)"""
    
    def __init__(self):
        self.models = {
            'quality_predictor': None,
            'anomaly_detector': None,
            'maintenance_predictor': None
        }
        self.scalers = {}
        self.performance_metrics = {}
        
    def train_quality_predictor(self, X_train, y_train):
        """품질 예측 모델 학습"""
        print("Training Quality Predictor...")
        
        # 스케일링
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_train)
        self.scalers['quality'] = scaler
        
        # Random Forest 모델
        model = RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            min_samples_split=5,
            random_state=42,
            n_jobs=-1
        )
        
        model.fit(X_scaled, y_train)
        self.models['quality_predictor'] = model
        
        # Feature importance
        feature_importance = pd.DataFrame({
            'feature': X_train.columns,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=False).head(10)
        
        print("  Top 10 Important Features:")
        for _, row in feature_importance.iterrows():
            print(f"    {row['feature']}: {row['importance']:.4f}")
        
        return model
    
    def train_anomaly_detector(self, X_normal):
        """이상 탐지 모델 학습"""
        print("\nTraining Anomaly Detector...")
        
        # 스케일링
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_normal)
        self.scalers['anomaly'] = scaler
        
        # Isolation Forest
        model = IsolationForest(
            contamination=0.05,
            random_state=42,
            n_jobs=-1
        )
        
        model.fit(X_scaled)
        self.models['anomaly_detector'] = model
        
        # 이상 점수 분포
        anomaly_scores = model.score_samples(X_scaled)
        print(f"  Anomaly score range: [{np.min(anomaly_scores):.3f}, {np.max(anomaly_scores):.3f}]")
        print(f"  Anomaly threshold: {np.percentile(anomaly_scores, 5):.3f}")
        
        return model
    
    def train_maintenance_predictor(self, X_train, y_maintenance):
        """예지보전 모델 (RUL 예측)"""
        print("\nTraining Maintenance Predictor...")
        
        # 간단한 RUL (Remaining Useful Life) 예측 모델
        # Tool wear를 기반으로 RUL 계산
        class MaintenanceModel:
            def __init__(self):
                self.threshold = 0.8
                
            def predict_rul(self, tool_wear, vibration):
                # 간단한 규칙 기반 RUL 예측
                base_rul = (1.0 - tool_wear) * 100  # 기본 RUL (시간)
                
                # 진동이 높으면 RUL 감소
                vibration_factor = np.clip(1.0 - (vibration - 2.0) / 3.0, 0.5, 1.0)
                
                rul = base_rul * vibration_factor
                
                return rul
            
            def predict_maintenance(self, tool_wear, vibration):
                rul = self.predict_rul(tool_wear, vibration)
                
                if rul < 10:
                    return 'urgent'
                elif rul < 30:
                    return 'scheduled'
                else:
                    return 'normal'
        
        model = MaintenanceModel()
        self.models['maintenance_predictor'] = model
        
        print("  Maintenance model configured")
        print("  RUL categories: urgent (<10h), scheduled (10-30h), normal (>30h)")
        
        return model
    
    def integrated_prediction(self, X):
        """통합 예측 수행"""
        results = {}
        
        # 1. 품질 예측
        if self.models['quality_predictor'] and 'quality' in self.scalers:
            X_scaled = self.scalers['quality'].transform(X)
            quality_pred = self.models['quality_predictor'].predict(X_scaled)
            quality_proba = self.models['quality_predictor'].predict_proba(X_scaled)
            results['quality_prediction'] = quality_pred
            results['quality_probability'] = quality_proba
        
        # 2. 이상 탐지
        if self.models['anomaly_detector'] and 'anomaly' in self.scalers:
            X_scaled = self.scalers['anomaly'].transform(X)
            anomaly_pred = self.models['anomaly_detector'].predict(X_scaled)
            anomaly_score = self.models['anomaly_detector'].score_samples(X_scaled)
            results['anomaly_detection'] = (anomaly_pred == -1).astype(int)
            results['anomaly_score'] = anomaly_score
        
        # 3. 예지보전
        if self.models['maintenance_predictor']:
            if 'tool_wear' in X.columns and 'vibration' in X.columns:
                maintenance_status = []
                rul_values = []
                
                for _, row in X.iterrows():
                    rul = self.models['maintenance_predictor'].predict_rul(
                        row['tool_wear'], row['vibration']
                    )
                    status = self.models['maintenance_predictor'].predict_maintenance(
                        row['tool_wear'], row['vibration']
                    )
                    rul_values.append(rul)
                    maintenance_status.append(status)
                
                results['maintenance_status'] = maintenance_status
                results['rul_hours'] = rul_values
        
        return results

# AI 시스템 구축
ai_system = IntegratedAISystem()

# 특징 선택
feature_cols = [col for col in train_data.columns 
                if col not in ['timestamp', 'defect', 'anomaly']]

X_train = train_data[feature_cols]
y_train = train_data['defect']

X_test = test_data[feature_cols]
y_test = test_data['defect']

# 모델 학습
print("\n" + "="*50)
print("TRAINING INTEGRATED AI SYSTEM")
print("="*50)

# 1. 품질 예측 모델
quality_model = ai_system.train_quality_predictor(X_train, y_train)

# 2. 이상 탐지 모델 (정상 데이터만 사용)
X_normal = X_train[train_data['anomaly'] == 0]
anomaly_model = ai_system.train_anomaly_detector(X_normal)

# 3. 예지보전 모델
maintenance_model = ai_system.train_maintenance_predictor(X_train, None)

print("\n✅ All models trained successfully!")

## 3. 통합 대시보드 및 모니터링

In [ ]:
class SmartFactoryDashboard:
    """스마트 팩토리 통합 대시보드"""
    
    def __init__(self, ai_system):
        self.ai_system = ai_system
        self.kpi_metrics = {}
        self.alerts = []
        
    def calculate_kpis(self, predictions, actuals=None):
        """주요 KPI 계산"""
        
        # 품질 KPI
        quality_pred = predictions.get('quality_prediction', [])
        if len(quality_pred) > 0:
            self.kpi_metrics['defect_rate'] = np.mean(quality_pred)
            self.kpi_metrics['quality_score'] = 100 * (1 - np.mean(quality_pred))
        
        # 이상 탐지 KPI
        anomaly_pred = predictions.get('anomaly_detection', [])
        if len(anomaly_pred) > 0:
            self.kpi_metrics['anomaly_rate'] = np.mean(anomaly_pred)
            self.kpi_metrics['system_health'] = 100 * (1 - np.mean(anomaly_pred))
        
        # 예지보전 KPI
        maintenance_status = predictions.get('maintenance_status', [])
        if maintenance_status:
            urgent_count = sum(1 for s in maintenance_status if s == 'urgent')
            scheduled_count = sum(1 for s in maintenance_status if s == 'scheduled')
            
            self.kpi_metrics['urgent_maintenance'] = urgent_count
            self.kpi_metrics['scheduled_maintenance'] = scheduled_count
            self.kpi_metrics['maintenance_score'] = 100 * (1 - urgent_count / len(maintenance_status))
        
        # RUL 통계
        rul_values = predictions.get('rul_hours', [])
        if rul_values:
            self.kpi_metrics['avg_rul'] = np.mean(rul_values)
            self.kpi_metrics['min_rul'] = np.min(rul_values)
        
        # OEE 계산 (Overall Equipment Effectiveness)
        availability = 0.95  # 가정
        performance = 0.90   # 가정
        quality = self.kpi_metrics.get('quality_score', 100) / 100
        self.kpi_metrics['oee'] = availability * performance * quality * 100
        
        return self.kpi_metrics
    
    def generate_alerts(self, predictions, data):
        """경보 생성"""
        alerts = []
        
        # 품질 경보
        quality_pred = predictions.get('quality_prediction', [])
        for i, pred in enumerate(quality_pred):
            if pred == 1:  # 불량 예측
                alerts.append({
                    'timestamp': datetime.now(),
                    'type': 'QUALITY',
                    'severity': 'HIGH',
                    'message': f'Defect predicted at index {i}',
                    'action': 'Inspect product immediately'
                })
        
        # 이상 경보
        anomaly_pred = predictions.get('anomaly_detection', [])
        anomaly_scores = predictions.get('anomaly_score', [])
        for i, (pred, score) in enumerate(zip(anomaly_pred, anomaly_scores)):
            if pred == 1:
                alerts.append({
                    'timestamp': datetime.now(),
                    'type': 'ANOMALY',
                    'severity': 'CRITICAL' if score < -0.5 else 'HIGH',
                    'message': f'Anomaly detected (score: {score:.3f})',
                    'action': 'Check sensor readings and equipment'
                })
        
        # 유지보수 경보
        maintenance_status = predictions.get('maintenance_status', [])
        for i, status in enumerate(maintenance_status):
            if status == 'urgent':
                alerts.append({
                    'timestamp': datetime.now(),
                    'type': 'MAINTENANCE',
                    'severity': 'CRITICAL',
                    'message': f'Urgent maintenance required',
                    'action': 'Schedule immediate maintenance'
                })
            elif status == 'scheduled':
                alerts.append({
                    'timestamp': datetime.now(),
                    'type': 'MAINTENANCE',
                    'severity': 'MEDIUM',
                    'message': f'Maintenance needed soon',
                    'action': 'Schedule maintenance within 24 hours'
                })
        
        self.alerts.extend(alerts)
        return alerts
    
    def visualize_dashboard(self, data, predictions):
        """대시보드 시각화"""
        fig = plt.figure(figsize=(16, 10))
        
        # KPI 카드 (상단)
        gs = fig.add_gridspec(3, 4, hspace=0.3, wspace=0.3)
        
        # OEE
        ax1 = fig.add_subplot(gs[0, 0])
        ax1.text(0.5, 0.7, f"{self.kpi_metrics.get('oee', 0):.1f}%", 
                ha='center', va='center', fontsize=24, fontweight='bold')
        ax1.text(0.5, 0.3, 'OEE', ha='center', va='center', fontsize=12)
        ax1.set_xlim(0, 1)
        ax1.set_ylim(0, 1)
        ax1.axis('off')
        
        # Quality Score
        ax2 = fig.add_subplot(gs[0, 1])
        ax2.text(0.5, 0.7, f"{self.kpi_metrics.get('quality_score', 0):.1f}%", 
                ha='center', va='center', fontsize=24, fontweight='bold',
                color='green' if self.kpi_metrics.get('quality_score', 0) > 95 else 'orange')
        ax2.text(0.5, 0.3, 'Quality Score', ha='center', va='center', fontsize=12)
        ax2.axis('off')
        
        # System Health
        ax3 = fig.add_subplot(gs[0, 2])
        ax3.text(0.5, 0.7, f"{self.kpi_metrics.get('system_health', 0):.1f}%", 
                ha='center', va='center', fontsize=24, fontweight='bold',
                color='green' if self.kpi_metrics.get('system_health', 0) > 90 else 'red')
        ax3.text(0.5, 0.3, 'System Health', ha='center', va='center', fontsize=12)
        ax3.axis('off')
        
        # Avg RUL
        ax4 = fig.add_subplot(gs[0, 3])
        ax4.text(0.5, 0.7, f"{self.kpi_metrics.get('avg_rul', 0):.0f}h", 
                ha='center', va='center', fontsize=24, fontweight='bold')
        ax4.text(0.5, 0.3, 'Avg RUL', ha='center', va='center', fontsize=12)
        ax4.axis('off')
        
        # 센서 데이터 트렌드
        ax5 = fig.add_subplot(gs[1, :])
        sensor_cols = ['temperature', 'pressure', 'vibration']
        for col in sensor_cols:
            if col in data.columns:
                ax5.plot(data.index[-100:], data[col].iloc[-100:], label=col, alpha=0.7)
        ax5.set_xlabel('Time')
        ax5.set_ylabel('Sensor Value')
        ax5.set_title('Sensor Data Trends (Last 100 samples)')
        ax5.legend()
        ax5.grid(True, alpha=0.3)
        
        # 예측 결과
        ax6 = fig.add_subplot(gs[2, :2])
        quality_pred = predictions.get('quality_prediction', [])
        if len(quality_pred) > 0:
            ax6.plot(quality_pred[-100:], 'r-', alpha=0.5, label='Defect Prediction')
            ax6.fill_between(range(len(quality_pred[-100:])), 
                            0, quality_pred[-100:], alpha=0.3, color='red')
        ax6.set_xlabel('Sample')
        ax6.set_ylabel('Defect Probability')
        ax6.set_title('Quality Predictions')
        ax6.legend()
        ax6.grid(True, alpha=0.3)
        
        # 알람 히스토리
        ax7 = fig.add_subplot(gs[2, 2:])
        if self.alerts:
            alert_types = [a['type'] for a in self.alerts[-20:]]
            alert_counts = pd.Series(alert_types).value_counts()
            colors = {'QUALITY': 'orange', 'ANOMALY': 'red', 'MAINTENANCE': 'yellow'}
            bar_colors = [colors.get(t, 'gray') for t in alert_counts.index]
            ax7.bar(alert_counts.index, alert_counts.values, color=bar_colors)
        ax7.set_xlabel('Alert Type')
        ax7.set_ylabel('Count')
        ax7.set_title('Recent Alerts (Last 20)')
        ax7.grid(True, alpha=0.3)
        
        plt.suptitle('Smart Factory Dashboard', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()

# 대시보드 실행
dashboard = SmartFactoryDashboard(ai_system)

# 통합 예측 수행
print("\n" + "="*50)
print("INTEGRATED PREDICTION & MONITORING")
print("="*50)

predictions = ai_system.integrated_prediction(X_test)

# KPI 계산
kpis = dashboard.calculate_kpis(predictions)
print("\nKey Performance Indicators:")
for key, value in kpis.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.2f}")
    else:
        print(f"  {key}: {value}")

# 알람 생성
alerts = dashboard.generate_alerts(predictions, test_data)
print(f"\nAlerts Generated: {len(alerts)}")
if alerts:
    print("  Recent Alerts:")
    for alert in alerts[:5]:
        print(f"    [{alert['severity']}] {alert['type']}: {alert['message']}")

# 대시보드 시각화
dashboard.visualize_dashboard(test_data, predictions)

## 4. 운영 계획 수립

In [ ]:
class OperationalPlan:
    """AI 시스템 운영 계획"""
    
    def __init__(self):
        self.deployment_plan = None
        self.maintenance_schedule = None
        self.monitoring_plan = None
        self.training_plan = None
        
    def create_deployment_plan(self):
        """배포 계획 수립"""
        self.deployment_plan = {
            'phase1': {
                'name': 'Pilot Testing',
                'duration': '2 weeks',
                'scope': 'Single production line',
                'activities': [
                    'Install sensors and edge devices',
                    'Configure data pipeline',
                    'Deploy AI models in shadow mode',
                    'Validate predictions against actual results'
                ],
                'success_criteria': {
                    'accuracy': '>90%',
                    'false_positive_rate': '<5%',
                    'system_uptime': '>99%'
                }
            },
            'phase2': {
                'name': 'Limited Production',
                'duration': '1 month',
                'scope': '3 production lines',
                'activities': [
                    'Scale to multiple lines',
                    'Enable real-time alerts',
                    'Train operators',
                    'Fine-tune models'
                ]
            },
            'phase3': {
                'name': 'Full Production',
                'duration': 'Ongoing',
                'scope': 'All production lines',
                'activities': [
                    'Full deployment',
                    'Integration with MES/ERP',
                    'Automated decision making',
                    'Continuous improvement'
                ]
            }
        }
        
        return self.deployment_plan
    
    def create_monitoring_plan(self):
        """모니터링 계획 수립"""
        self.monitoring_plan = {
            'real_time_monitoring': {
                'metrics': [
                    'Prediction accuracy',
                    'Inference latency',
                    'Data quality',
                    'System health'
                ],
                'tools': ['Grafana', 'Prometheus', 'ELK Stack'],
                'alert_channels': ['Email', 'SMS', 'Slack']
            },
            'model_monitoring': {
                'drift_detection': 'Weekly',
                'performance_evaluation': 'Daily',
                'feature_importance_check': 'Monthly'
            },
            'business_monitoring': {
                'kpi_review': 'Weekly',
                'roi_calculation': 'Monthly',
                'stakeholder_reporting': 'Monthly'
            }
        }
        
        return self.monitoring_plan
    
    def create_maintenance_schedule(self):
        """유지보수 일정 수립"""
        self.maintenance_schedule = {
            'daily': [
                'Check system health dashboard',
                'Review alert logs',
                'Validate data pipeline'
            ],
            'weekly': [
                'Model performance review',
                'Data quality assessment',
                'Backup model artifacts',
                'Update documentation'
            ],
            'monthly': [
                'Model retraining evaluation',
                'Feature engineering review',
                'System optimization',
                'Security audit'
            ],
            'quarterly': [
                'Major model updates',
                'Infrastructure scaling review',
                'Disaster recovery test',
                'Compliance check'
            ]
        }
        
        return self.maintenance_schedule
    
    def create_training_plan(self):
        """교육 계획 수립"""
        self.training_plan = {
            'operators': {
                'duration': '2 days',
                'topics': [
                    'Dashboard navigation',
                    'Alert interpretation',
                    'Basic troubleshooting',
                    'Data entry procedures'
                ]
            },
            'engineers': {
                'duration': '1 week',
                'topics': [
                    'System architecture',
                    'Model interpretation',
                    'Parameter tuning',
                    'Advanced troubleshooting'
                ]
            },
            'data_scientists': {
                'duration': '2 weeks',
                'topics': [
                    'Model development',
                    'Feature engineering',
                    'MLOps practices',
                    'Continuous improvement'
                ]
            },
            'management': {
                'duration': '1 day',
                'topics': [
                    'KPI interpretation',
                    'ROI analysis',
                    'Strategic planning',
                    'Change management'
                ]
            }
        }
        
        return self.training_plan
    
    def calculate_roi(self, investment=100000, monthly_savings=15000, months=24):
        """ROI 계산"""
        total_savings = monthly_savings * months
        roi = ((total_savings - investment) / investment) * 100
        payback_period = investment / monthly_savings
        
        return {
            'initial_investment': investment,
            'monthly_savings': monthly_savings,
            'total_savings': total_savings,
            'roi_percentage': roi,
            'payback_months': payback_period
        }

# 운영 계획 수립
ops_plan = OperationalPlan()

print("\n" + "="*50)
print("OPERATIONAL PLANNING")
print("="*50)

# 배포 계획
deployment_plan = ops_plan.create_deployment_plan()
print("\n📋 Deployment Plan:")
for phase, details in deployment_plan.items():
    print(f"\n  {phase.upper()}: {details['name']}")
    print(f"    Duration: {details['duration']}")
    print(f"    Scope: {details['scope']}")

# 모니터링 계획
monitoring_plan = ops_plan.create_monitoring_plan()
print("\n📊 Monitoring Plan:")
for category, details in monitoring_plan.items():
    print(f"  {category.replace('_', ' ').title()}:")
    for key, value in details.items():
        if isinstance(value, list):
            print(f"    {key}: {', '.join(value)}")
        else:
            print(f"    {key}: {value}")

# ROI 계산
roi_calc = ops_plan.calculate_roi(
    investment=150000,
    monthly_savings=25000,
    months=24
)

print("\n💰 ROI Analysis:")
print(f"  Initial Investment: ${roi_calc['initial_investment']:,}")
print(f"  Monthly Savings: ${roi_calc['monthly_savings']:,}")
print(f"  Total Savings (24 months): ${roi_calc['total_savings']:,}")
print(f"  ROI: {roi_calc['roi_percentage']:.1f}%")
print(f"  Payback Period: {roi_calc['payback_months']:.1f} months")

## 5. 체크리스트 및 다음 단계

In [ ]:
# 구현 체크리스트
checklist = {
    'Data Infrastructure': [
        '✓ Sensor data collection pipeline',
        '✓ Data quality validation',
        '✓ Feature engineering pipeline',
        '✓ Data storage and backup'
    ],
    'AI Models': [
        '✓ Quality prediction model',
        '✓ Anomaly detection model',
        '✓ Predictive maintenance model',
        '✓ Model versioning system'
    ],
    'Deployment': [
        '✓ API endpoints',
        '✓ Model serving infrastructure',
        '□ Container orchestration',
        '□ Load balancing'
    ],
    'Monitoring': [
        '✓ Real-time dashboard',
        '✓ Alert system',
        '✓ KPI tracking',
        '□ Drift detection automation'
    ],
    'Operations': [
        '✓ Deployment plan',
        '✓ Maintenance schedule',
        '□ Training materials',
        '□ Documentation'
    ]
}

print("\n" + "="*50)
print("IMPLEMENTATION CHECKLIST")
print("="*50)

for category, items in checklist.items():
    print(f"\n{category}:")
    for item in items:
        print(f"  {item}")

# 다음 단계
next_steps = [
    "1. Pilot deployment on selected production line",
    "2. Collect feedback from operators and engineers",
    "3. Fine-tune models based on real production data",
    "4. Develop automated retraining pipeline",
    "5. Scale to additional production lines",
    "6. Integrate with existing MES/ERP systems",
    "7. Implement advanced features (multi-variate analysis, root cause analysis)",
    "8. Establish continuous improvement process"
]

print("\n" + "="*50)
print("NEXT STEPS")
print("="*50)

for step in next_steps:
    print(f"  {step}")

print("\n" + "="*50)
print("🎯 PROJECT SUMMARY")
print("="*50)
print("""
Smart Factory AI System Successfully Designed!

✅ Integrated AI system with 3 core models
✅ Real-time monitoring dashboard
✅ Comprehensive alert system
✅ Operational planning framework
✅ ROI analysis completed

Ready for pilot deployment!
""")

## 실습 과제

### 개인 프로젝트 정의
1. **문제 정의**: 자신의 제조 현장에서 해결하고자 하는 구체적인 문제 선택
2. **데이터 수집 계획**: 필요한 데이터와 수집 방법 정의
3. **모델 선택**: 적합한 AI 모델 아키텍처 선택
4. **성공 지표**: 측정 가능한 KPI 정의
5. **구현 로드맵**: 3-6개월 구현 계획 수립

### 팀 프로젝트
1. **Cross-functional 팀 구성**: IT, 생산, 품질 부서 협업
2. **PoC 프로젝트 선정**: Quick-win 가능한 과제 선택
3. **리스크 분석**: 기술적/운영적 리스크 식별
4. **변경 관리 계획**: 조직 변화 관리 전략

### 지속적 학습
1. **기술 트렌드 모니터링**: 최신 제조 AI 기술 동향
2. **커뮤니티 참여**: 제조 AI 커뮤니티 활동
3. **내부 교육 프로그램**: 조직 내 AI 역량 확산
4. **벤치마킹**: 선진 사례 연구 및 적용